In [78]:
import plotly.express as px
from sqlalchemy import create_engine, engine
import pandas as pd
import numpy as np
import codecs
import re


In [86]:
data = ["2023-05-22 06:45:34.22222", "2019-05-22 15:45:34.22222", "2015-05-22 20:45:34.22222", "2010-05-22 14:45:34.22222"]
riders_day_df = pd.DataFrame(data, columns=['start_time'])
total = riders_day_df["start_time"].count()
print_line = f"Total number of riders in the last 24 hours: {total}"


In [80]:
data = [["2023-05-22 06:45:34.22222", "male"], ["2019-05-22 15:45:34.22222", "male"], ["2015-05-22 20:45:34.22222", "female"], ["2010-05-22 14:45:34.22222", "male"]]
riders_day_gender_df = pd.DataFrame(data, columns=['start_time', 'gender'])
gender_count = riders_day_gender_df.groupby('gender')['start_time'].count().reset_index(name='count')
fig = px.bar(gender_count, x = 'gender', y = 'count', title="Gender split between riders in the last 24 hours")
fig.update_layout(
    xaxis_title='Gender',
    yaxis_title='Count'
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'gender=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['female', 'male'], dtype=object),
              'xaxis': 'x',
              'y': array([1, 3]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Gender split between riders in the last 24 hours'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Gender'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Count'}}}
})

In [81]:
data = [["2023-05-16 06:45:34.22222", "2000-04-22"], ["2023-05-16 15:45:34.22222", "1992-08-10"], ["2023-05-16 20:45:34.22222", "2004-06-14"], ["2023-05-16 14:45:34.22222", "1999-12-25"]]
riders_day_gender_df = pd.DataFrame(data, columns=['start_time', 'dob'])
riders_day_gender_df['start_time'] = pd.to_datetime(riders_day_gender_df['start_time'])
riders_day_gender_df['dob'] = pd.to_datetime(riders_day_gender_df['dob'])
riders_day_gender_df['age'] = ((riders_day_gender_df['start_time'] - riders_day_gender_df['dob']).dt.days.astype(float)) * 0.00273973 #converts to normal number
riders_day_gender_df['age'] = riders_day_gender_df['age'].apply(np.floor)
print(riders_day_gender_df)

                  start_time        dob   age
0 2023-05-16 06:45:34.222220 2000-04-22  23.0
1 2023-05-16 15:45:34.222220 1992-08-10  30.0
2 2023-05-16 20:45:34.222220 2004-06-14  18.0
3 2023-05-16 14:45:34.222220 1999-12-25  23.0


In [82]:
bins = [10, 20, 30, 40, 50, 60, 70, 105]
labels = ['10-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71+']
age_groups = pd.cut(riders_day_gender_df['age'], bins=bins, labels=labels, right=True)
age_group_count = riders_day_gender_df.groupby([age_groups])['start_time'].count().reset_index(name='count')
age_fig = px.bar(age_group_count, x = 'age', y = 'count', title="Age group split between riders in the last 24 hours")
age_fig.update_layout(
    xaxis_title='Age',
    yaxis_title='Count'
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'age=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['10-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71+'],
                         dtype=object),
              'xaxis': 'x',
              'y': array([1, 3, 0, 0, 0, 0, 0]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Age group split between riders in the last 24 hours'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Age'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Count'}}}
})

In [90]:
data = [["2023-05-16 06:45:34.22222", "84", "11.22896664"], ["2023-05-16 15:45:34.22222", "85", "12.54785501"], ["2023-05-16 20:45:34.22222", "63", "1.648772319"], ["2023-05-16 14:45:34.22222", "72", "8.962429663"]]
riders_day_reading_df = pd.DataFrame(data, columns=['start_time', 'avg_heart_rate', 'avg_power'])
heart_fig = px.scatter(riders_day_reading_df, x="start_time", y="avg_heart_rate", color="avg_heart_rate", title="Average heart rate in the past day")
heart_fig.update_layout(
xaxis_title ="Time",
yaxis_title ="Heart rate"
)
power_fig = px.scatter(riders_day_reading_df, x="start_time", y="avg_power", color="avg_power", title="Average power in the past day")
power_fig.update_layout(
xaxis_title ="Time",
yaxis_title ="Power"
)
with open('report_graph.html', 'w') as f:
    f.write(f"<html>\n<head>\n<title>Daily Report</title>\n</head>\n<body>\n<h1>Daily report</h1>\n<p1>{print_line}</p1>")
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(age_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(heart_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(power_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write("\n</body>\n</html>")

print(print_line)

Total number of riders in the last 24 hours: 4
